In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import json
import datetime

## Importing data

In [2]:
matches = pd.read_csv("matches.csv")
matches.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode
0,2015-03-04,AFG,AUS,Away,3773,65
1,2012-08-25,AFG,AUS,Neutral,3437,156
2,2019-06-01,AFG,AUS,Neutral,4306,149
3,2014-03-01,AFG,BAN,Away,3623,190
4,2016-09-25,AFG,BAN,Away,3936,198


In [3]:
attributes = pd.read_csv("attributes.csv")
attributes.head()

,Full Name,Born,Bats,Bowls,Code
0,David A Warner,1986-10-27,Left Handed,Right Arm Leg Break,3637
1,Aaron James Finch,1986-11-17,Right Handed,Left Arm Orthodox,3845
2,Steven Peter Devereux Smith,1989-02-06,Right Handed,Right Arm Leg Break,3756
3,Glenn James Maxwell,1988-10-14,Right Handed,Right Arm Off Break,3969
4,James Peter Faulkner,1990-04-29,Right Handed,Left Arm Medium,3930


In [4]:
batsmen = open('batsmen.json',) 
batsmen_data = json.load(batsmen)

In [5]:
bowlers = open('bowlers.json',) 
bowlers_data = json.load(bowlers)

In [6]:
invtmap = open('invtmap.json',) 
invtmap_data = json.load(invtmap)

In [7]:
scorecard = open('scorecard.json',) 
scorecard_data = json.load(scorecard)

In [8]:
region = open('region.json',) 
region_data = json.load(region)

In [9]:
tmap = open('tmap.json',) 
tmap_data = json.load(tmap)

## Making the Database

In [10]:
countries = list(set(matches['Team_1'].tolist()) | set(matches['Team_2'].tolist()))
countries

['KEN',
 'IRE',
 'NED',
 'BAN',
 'ENG',
 'SAF',
 'SCO',
 'WIN',
 'NZL',
 'IND',
 'ZIM',
 'AFG',
 'PAK',
 'SRL',
 'AUS']

In [11]:
ground = list(set(matches['GroundCode'].tolist()) )
#ground

In [12]:
#Input
team_1 = 'IND'
team_2 = 'ZIM' 
venue = 26
date = '2015-07-12'

match_1 = matches[matches['Date']==date]
match = match_1[match_1['Team_1']==team_1]
match_code = match['MatchCode'].tolist()[0]
score_card = scorecard_data[str(match_code)]
team_1_players = [score_card['BATTING1'][i][0] for i in range(10)]
team_2_players = [score_card['BATTING2'][i][0] for i in range(10)]
#name
team_1_players_name = [attributes[attributes['Code']==team_1_players[i]].iloc[0]['Full Name'] for i in range(10)]
team_2_players_name = [attributes[attributes['Code']==team_2_players[i]].iloc[0]['Full Name'] for i in range(10)]
#Batting style
team_1_players_bats = [attributes[attributes['Code']==team_1_players[i]].iloc[0]['Bats'] for i in range(10)]
team_2_players_bats = [attributes[attributes['Code']==team_2_players[i]].iloc[0]['Bats'] for i in range(10)]
#Bowling style
team_1_players_bowls = [attributes[attributes['Code']==team_1_players[i]].iloc[0]['Bowls'] for i in range(10)]
team_2_players_bowls = [attributes[attributes['Code']==team_2_players[i]].iloc[0]['Bowls'] for i in range(10)]
#Form for each player


In [13]:
years_to_take_in = 5 #number of years to go back for dataset
matches_needed_for_dataset = matches[matches['Team_1']==team_1]
matches_needed_for_dataset = matches_needed_for_dataset[matches_needed_for_dataset['Team_2']==team_2]
matches_needed_for_dataset['Date']= pd.to_datetime(matches_needed_for_dataset['Date'])
date_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
date_obj_min = datetime.datetime.strptime(str(date_obj.year-years_to_take_in)+'-01'+'-01', '%Y-%m-%d')
matches_needed_for_dataset = matches_needed_for_dataset[matches_needed_for_dataset['Date']>=date_obj_min]
Dataset = matches_needed_for_dataset.reset_index()
MatchCodes = Dataset['MatchCode'].tolist()
Dataset = Dataset.drop(columns=['index'])

In [20]:
Dataset['Toss'] = ''
Dataset['Fours_team1'] = ''
Dataset['Fours_team2'] = ''
Dataset['Sixes_team1'] = ''
Dataset['Sixes_team2'] = ''
Dataset['Strike_rate_team1'] = ''
Dataset['Strike_rate_team2'] = ''
Dataset['Maidens_team1'] = ''
Dataset['Maidens_team2'] = ''
Dataset['Wickets_team1'] = ''
Dataset['Wickets_team2'] = ''
Dataset['Economy_rate_team1'] = ''
Dataset['Economy_rate_team2'] = ''
Dataset['Result'] = ''

In [21]:
for i in range(len(MatchCodes)):
    matchdata = scorecard_data[str(MatchCodes[i])]
    #Dataset = Dataset.drop(columns=['GroundCode','Team_1','Team_2'])
    if (matchdata['TOSS']==matchdata['ORDER'][0]):
        toss = team_1
    else:
        toss = team_2
    #Batting 1
    batting1 = matchdata['BATTING1']
    Runs_team1 = []
    Balls_Played_team1 = []
    Fours_team1 = []
    Sixes_team1 = []
    Strike_rate_team1 = []
    for val in range(10):
        Runs_team1.append(batting1[val][2])
        Balls_Played_team1.append(batting1[val][3])
        Fours_team1.append(batting1[val][4])
        Sixes_team1.append(batting1[val][5])
        Strike_rate_team1.append(batting1[val][6])
        
    #Batting 2
    batting2 = matchdata['BATTING2']
    Runs_team2 = []
    Balls_Played_team2 = []
    Fours_team2 = []
    Sixes_team2 = []
    Strike_rate_team2 = []
    for val in range(10):
        Runs_team2.append(batting2[val][2])
        Balls_Played_team2.append(batting2[val][3])
        Fours_team2.append(batting2[val][4])
        Sixes_team2.append(batting2[val][5])
        Strike_rate_team2.append(batting2[val][6])

    #Bowling 1
    bowling1 = matchdata['BOWLING1']
    Overs_team1 = []
    Maidens_team1 = []
    Runs_given_team1 = []
    Wickets_team1 = []
    Economy_rate_team1 = []
    for val in range(10):
        if val<len(bowling1):
            Overs_team1.append(bowling1[val][1])
            Maidens_team1.append(bowling1[val][2])
            Runs_given_team1.append(bowling1[val][3])
            Wickets_team1.append(bowling1[val][4])
            Economy_rate_team1.append(bowling1[val][5])
        else:
            Overs_team1.append(-1)
            Maidens_team1.append(-1)
            Runs_given_team1.append(-1)
            Wickets_team1.append(-1)
            Economy_rate_team1.append(-1)

    #Bowling 2
    bowling2 = matchdata['BOWLING2']
    Overs_team2 = []
    Maidens_team2 = []
    Runs_given_team2 = []
    Wickets_team2 = []
    Economy_rate_team2 = []
    for val in range(10):
        if val<len(bowling2):
            Overs_team2.append(bowling2[val][1])
            Maidens_team2.append(bowling2[val][2])
            Runs_given_team2.append(bowling2[val][3])
            Wickets_team2.append(bowling2[val][4])
            Economy_rate_team2.append(bowling2[val][5])
        else:
            Overs_team2.append(-1)
            Maidens_team2.append(-1)
            Runs_given_team2.append(-1)
            Wickets_team2.append(-1)
            Economy_rate_team2.append(-1)
    #Scores
    score = matchdata['SCORES']
    if (score[0]>score[1]):
        result = team_1
    elif (score[1]>score[0]):
        result = team_2
    else:
        result = 'Tie'
    
    #insert into dataset
    Dataset['Toss'].iloc[i] = toss
    Dataset['Fours_team1'].iloc[i] = Fours_team1
    Dataset['Fours_team2'].iloc[i] = Fours_team2
    Dataset['Sixes_team1'].iloc[i] = Sixes_team1
    Dataset['Sixes_team2'].iloc[i] = Sixes_team2
    Dataset['Strike_rate_team1'].iloc[i] = Strike_rate_team1
    Dataset['Strike_rate_team2'].iloc[i] = Strike_rate_team2
    Dataset['Maidens_team1'].iloc[i] = Maidens_team1
    Dataset['Maidens_team2'].iloc[i] = Maidens_team2
    Dataset['Wickets_team1'].iloc[i] = Wickets_team1
    Dataset['Wickets_team2'].iloc[i] = Wickets_team2
    Dataset['Economy_rate_team1'].iloc[i] = Economy_rate_team1
    Dataset['Economy_rate_team2'].iloc[i] = Economy_rate_team2
    Dataset['Result'].iloc[i] = result

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
Dataset.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,Toss,Fours_team1,Fours_team2,Sixes_team1,Sixes_team2,Strike_rate_team1,Strike_rate_team2,Maidens_team1,Maidens_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2,Result
0,2010-05-28,IND,ZIM,Away,3111,73,IND,"[2.0, 1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1]","[1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1]","[0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1, -1]","[56.41, 45.83, 0.0, 95.8, 69.81, 100.0, 220.0,...","[45.83, 0.0, 95.8, 69.81, 100.0, 220.0, -1, -1...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1]","[6.3, 18.0, 5.3, 4.6, 4.0, 5.71, 5.0, -1, -1, -1]","[6.68, 6.38, 6.0, 4.7, 5.1, 7.0, 5.0, -1, -1, -1]",ZIM
1,2010-06-03,IND,ZIM,Away,3116,26,ZIM,"[4.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, ...","[70.21, 37.5, 62.07, 52.0, 60.0, 70.83, 60.0, ...","[37.5, 62.07, 52.0, 60.0, 70.83, 60.0, 0.0, 94...","[1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 1.0, 3.0, 1.0, 1.0, -1, -1, -1, -1, -1]","[0.0, 1.0, 0.0, 0.0, 2.0, -1, -1, -1, -1, -1]","[2.2, 3.7, 4.5, 4.1, 4.2, -1, -1, -1, -1, -1]","[6.48, 3.5, 5.0, 6.22, 3.86, -1, -1, -1, -1, -1]",ZIM
2,2013-07-24,IND,ZIM,Away,3537,26,ZIM,"[3.0, 6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1]","[6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1, -1]","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1]","[2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1, -1]","[47.22, 73.21, 51.72, 91.67, 80.0, 28.57, 126....","[73.21, 51.72, 91.67, 80.0, 28.57, 126.47, 114...","[1.0, 1.0, 1.0, 3.0, 1.0, 0.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[1.0, 1.0, 1.0, 0.0, 3.0, 1.0, -1, -1, -1, -1]","[1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, -1, -1, -1]","[6.33, 4.5, 3.9, 3.3, 4.3, 2.0, -1, -1, -1, -1]","[5.0, 3.33, 6.0, 3.4, 8.3, 6.0, 6.0, -1, -1, -1]",ZIM
3,2013-07-26,IND,ZIM,Away,3539,26,ZIM,"[0.0, 11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0,...","[11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0, 0.0,...","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, ...","[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, -1]","[14.29, 91.34, 77.78, 20.0, 36.36, 93.24, 68.1...","[91.34, 77.78, 20.0, 36.36, 93.24, 68.18, 128....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 0.0, 3.0, 1.0, 1.0, 0.0, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 0.0, 1.0, -1, -1, -1]","[0.0, 1.0, 4.0, 1.0, 2.0, 0.0, -1, -1, -1, -1]","[7.0, 3.38, 5.11, 5.0, 6.7, 3.75, 11.33, -1, -...","[5.44, 5.2, 4.1, 3.0, 4.6, 5.0, -1, -1, -1, -1]",IND
4,2013-07-28,IND,ZIM,Away,3541,26,ZIM,"[0.0, 0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, ...","[0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[0.0, 33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26...","[33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26.32, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, -1, -1, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 0.0, 4.0, -1, -1, -1]","[1.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[4.57, 2.78, 3.43, 7.0, 3.9, 2.0, 4.7, -1, -1,...","[6.63, 5.14, 4.53, 4.1, 7.0, -1, -1, -1, -1, -1]",ZIM


In [23]:
Dataset = Dataset.drop(columns=['Date','Team_1','Team_2','MatchCode','GroundCode'])
Dataset

,Venue,Toss,Fours_team1,Fours_team2,Sixes_team1,Sixes_team2,Strike_rate_team1,Strike_rate_team2,Maidens_team1,Maidens_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2,Result
0,Away,IND,"[2.0, 1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1]","[1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1]","[0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1, -1]","[56.41, 45.83, 0.0, 95.8, 69.81, 100.0, 220.0,...","[45.83, 0.0, 95.8, 69.81, 100.0, 220.0, -1, -1...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1]","[6.3, 18.0, 5.3, 4.6, 4.0, 5.71, 5.0, -1, -1, -1]","[6.68, 6.38, 6.0, 4.7, 5.1, 7.0, 5.0, -1, -1, -1]",ZIM
1,Away,ZIM,"[4.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, ...","[70.21, 37.5, 62.07, 52.0, 60.0, 70.83, 60.0, ...","[37.5, 62.07, 52.0, 60.0, 70.83, 60.0, 0.0, 94...","[1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 1.0, 3.0, 1.0, 1.0, -1, -1, -1, -1, -1]","[0.0, 1.0, 0.0, 0.0, 2.0, -1, -1, -1, -1, -1]","[2.2, 3.7, 4.5, 4.1, 4.2, -1, -1, -1, -1, -1]","[6.48, 3.5, 5.0, 6.22, 3.86, -1, -1, -1, -1, -1]",ZIM
2,Away,ZIM,"[3.0, 6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1]","[6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1, -1]","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1]","[2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1, -1]","[47.22, 73.21, 51.72, 91.67, 80.0, 28.57, 126....","[73.21, 51.72, 91.67, 80.0, 28.57, 126.47, 114...","[1.0, 1.0, 1.0, 3.0, 1.0, 0.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[1.0, 1.0, 1.0, 0.0, 3.0, 1.0, -1, -1, -1, -1]","[1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, -1, -1, -1]","[6.33, 4.5, 3.9, 3.3, 4.3, 2.0, -1, -1, -1, -1]","[5.0, 3.33, 6.0, 3.4, 8.3, 6.0, 6.0, -1, -1, -1]",ZIM
3,Away,ZIM,"[0.0, 11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0,...","[11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0, 0.0,...","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, ...","[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, -1]","[14.29, 91.34, 77.78, 20.0, 36.36, 93.24, 68.1...","[91.34, 77.78, 20.0, 36.36, 93.24, 68.18, 128....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 0.0, 3.0, 1.0, 1.0, 0.0, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 0.0, 1.0, -1, -1, -1]","[0.0, 1.0, 4.0, 1.0, 2.0, 0.0, -1, -1, -1, -1]","[7.0, 3.38, 5.11, 5.0, 6.7, 3.75, 11.33, -1, -...","[5.44, 5.2, 4.1, 3.0, 4.6, 5.0, -1, -1, -1, -1]",IND
4,Away,ZIM,"[0.0, 0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, ...","[0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[0.0, 33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26...","[33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26.32, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, -1, -1, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 0.0, 4.0, -1, -1, -1]","[1.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[4.57, 2.78, 3.43, 7.0, 3.9, 2.0, 4.7, -1, -1,...","[6.63, 5.14, 4.53, 4.1, 7.0, -1, -1, -1, -1, -1]",ZIM
5,Away,ZIM,"[3.0, 1.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[53.33, 41.18, 66.67, 0.0, 0.0, 45.45, 75.76, ...","[41.18, 66.67, 0.0, 0.0, 45.45, 75.76, 8.33, 1...","[3.0, 1.0, 0.0, 1.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 1.0, 1.0, 2.0, 3.0, -1, -1, -1, -1, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.6, 4.25, 3.86, 3.11, 2.88, -1, -1, -1, -1, -1]","[3.6, 3.5, 6.2, 4.67, 6.0, 5.25, 3.0, -1, -1, -1]",ZIM
6,Away,ZIM,"[1.0, 4.0, 0.0, 0.0, 6.0, 0.0, 3.0, 0.0, 0.0, ..

### Moving Average